## 03 - Modelling

## Optional - Colab Setup

In [0]:
def upgrade_runtime_ram():
    meminfo = subprocess.getoutput('cat /proc/meminfo').split('\n')

    memory_info = {entry.split(':')[0]: int(entry.split(':')[1].replace(' kB','').strip()) for entry in meminfo}

    if memory_info['MemTotal'] > 17000000:
        return

    a = []
    while(1):
        a.append('1')

In [0]:
def restart_runtime():
    os.kill(os.getpid(), 9)

In [0]:
def setup_rapids():
    pynvml.nvmlInit()
    handle = pynvml.nvmlDeviceGetHandleByIndex(0)
    device_name = pynvml.nvmlDeviceGetName(handle)
    if (device_name != b'Tesla T4') and (device_name != b'Tesla P4') and (device_name != b'Tesla P100-PCIE-16GB'):
        print("Wrong GPU - Restarting Runtime")
        restart_runtime()


    # clone RAPIDS AI rapidsai-csp-utils scripts repo
    !git clone https://github.com/rapidsai/rapidsai-csp-utils.git

    # install RAPIDS
    !bash rapidsai-csp-utils/colab/rapids-colab.sh 0.13


    # set necessary environment variables 
    dist_package_index = sys.path.index('/usr/local/lib/python3.6/dist-packages')
    sys.path = sys.path[:dist_package_index] + ['/usr/local/lib/python3.6/site-packages'] + sys.path[dist_package_index:]
    sys.path

    # update pyarrow & modules 
    exec(open('rapidsai-csp-utils/colab/update_modules.py').read(), globals())

In [0]:
def setup_conda():
    if not 'Miniconda3-4.5.4-Linux-x86_64.sh' in os.listdir():
        !wget https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh && bash Miniconda3-4.5.4-Linux-x86_64.sh -bfp /usr/local

    if not ('EPFL-Capstone-Project' in os.listdir()) and (os.getcwd().split('/')[-1] != 'EPFL-Capstone-Project'):
        !git clone https://github.com/helmigsimon/EPFL-Capstone-Project  
    if 'EPFL-Capstone-Project' in os.listdir():
        os.chdir('EPFL-Capstone-Project')

    !conda env create -f environment.yml
    !conda activate exts-ml

In [0]:
def setup_drive():
    #Mounting Google Drive
    global drive
    from google.colab import drive
    drive.mount('/content/drive')

In [6]:
try:
    import sys,os,subprocess
    
    upgrade_runtime_ram()
    setup_drive()

    #Setting up PyPi Packages
    !pip install geopandas sparse-dot-topn pdpipe category-encoders
    import geopandas as gpd
    import sparse_dot_topn.sparse_dot_topn as ct
    import pdpipe as pdp
    import category_encoders

    #Setting up Conda Packages
    setup_conda()
    
    #Initializing NLTK
    import nltk
    nltk.download('stopwords')
    nltk.download('punkt')
    
    #Setting up RAPIDS AI
    import pynvml
    setup_rapids()
    
    from cuml import UMAP
    
except ModuleNotFoundError as e:
    print(e)
    print('Not in colab environment, continuing to run locally')
    from umap import UMAP

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  Using cached geopandas-0.7.0-py2.py3-none-any.whl (928 kB)
Processing /root/.cache/pip/wheels/aa/e7/c7/e64559d518bcc12ed7e40a605ae304259b957ae8181c8d1e82/sparse_dot_topn-0.2.9-cp36-cp36m-linux_x86_64.whl
  Using cached pdpipe-0.0.46-py3-none-any.whl (47 kB)
  Using cached category_encoders-2.1.0-py2.py3-none-any.whl (100 kB)
  Using cached Fiona-1.8.13.post1-cp36-cp36m-manylinux1_x86_64.whl (14.7 MB)
  Using cached pyproj-2.6.0-cp36-cp36m-manylinux2010_x86_64.whl (10.4 MB)
     |████████████████████████████████| 1.8 MB 2.7 MB/s 
     |████████████████████████████████| 2.1 MB 14.3 MB/s 
  Using cached strct-0.0.30-py2.py3-none-any.whl (16 kB)
  Using cached skutil-0.0.16-py2.py3-none-any.whl (18 kB)
     |████████████████████████████████| 60 kB 7.0 MB/s 
     |████████████████████████████████| 8.7 MB 150 kB/s 
     |████████████████████████████████| 231 kB 4

## Imports

In [0]:
%matplotlib inline
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split,  StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
tqdm.pandas()

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer

In [0]:
from lib.transformers import *
from lib.pipelines import *
from lib.processing import save_to_pkl, load_from_pkl
from data.util.paths import DATA_PATH

In [0]:
from category_encoders.leave_one_out import LeaveOneOutEncoder

## Setting up DataFrame

In [0]:
api_df = load_from_pkl('api',DATA_PATH)
extracted_df = load_from_pkl('extracted',DATA_PATH)

In [12]:
api_df = api_pipe.fit_transform(api_df)
extracted_df = extracted_pipe.fit_transform(extracted_df)

In [0]:
#Remove outliers from extracted_df
extracted_df = OutlierRemover('market_value').fit_transform(extracted_df)

## Reducing Dimensionality of Image Embeddings

In [0]:
with np.load(os.path.join(DATA_PATH,'high_level_features_labelled.npz')) as data:
    image_embedding_df = pd.concat([pd.DataFrame(data[section]) for section in ('release_id','bitmap','features')],axis=1)
    image_embedding_df.columns = ['release_id', 'bitmap'] + ['feature_%s' % i for i in range(1,1281)]

In [0]:
from cuml import UMAP

In [0]:
scaler = StandardScaler()
umap = UMAP(n_components=10)
image_embeddings_scaled = scaler.fit_transform(image_embedding_df.loc[:,['feature_%s' % i for i in range(1,1281)]])


In [17]:
image_embeddings_reduced = umap.fit_transform(image_embeddings_scaled)

/usr/local/lib/python3.6/site-packages/cuml/utils/input_utils.py:188: UserWarning: Expected row ('C') major order, but got the opposite. Converting data, this will result in additional memory utilization.
  warnings.warn("Expected " + order_to_str(order) + " major order, "


In [0]:
image_embeddings_reduced = pd.concat([
      image_embedding_df.loc[:,'release_id'],
      pd.DataFrame(
          image_embeddings_reduced,
          columns = ['images_umap_%s' % i for i in range(image_embeddings_reduced.shape[1])]
      )],
      axis=1
)


Combining datasets

In [0]:
df = api_df.merge(extracted_df,how='inner',on='release_id')
df = df.merge(image_embeddings_reduced,how='inner',on='release_id')

In [0]:
del api_df, extracted_df, image_embedding_df, image_embeddings_scaled, image_embeddings_reduced

In [21]:
col_set = {
    'format': {
        'description': 'format_description_', 
        'name': 'format_name_', 
        'text': ('format_text_clean'),
        'quantity': ('format_quantity')
    },
    'geography': {
        'superregion': 'superregion_',
        'region': 'region_',
        'country': 'country_'
    },
    'timeperiod': {
        'period': 'period_',
        'era': 'era_'
    },
    'genre': 'genre_',
    'style': 'style_',
    'null': None,
    'indicator': lambda x: x.max() == 1 and x.min() == 0,
    'image_embedding': 'image_umap_'
}
column_store = ColumnStore()
column_store.fit(df,col_set)

## Preparing for Machine Learning

In [0]:
X_tr, X_te, y_tr, y_te = train_test_split(df.drop('market_value',axis=1),df.market_value)

In [0]:
del df

## Setting up Dummy Regression for baseline

In [0]:
from sklearn.dummy import DummyRegressor

In [0]:
dummy_val = GridSearchCV(
    DummyRegressor(),
    cv=5,
    param_grid = {
        'strategy': ['mean']
    },
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
    verbose=5
)

In [26]:
dummy_val.fit(X_tr,np.log(y_tr).values)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   14.0s remaining:   20.9s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   25.5s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=DummyRegressor(constant=None, quantile=None,
                                      strategy='mean'),
             iid='deprecated', n_jobs=-1, param_grid={'strategy': ['mean']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_absolute_error', verbose=5)

In [27]:
dummy_val.best_score_

-0.6975033111515884

In [0]:
y_mean_pred = np.full(shape=y_te.shape,fill_value=np.mean(y_te))

In [29]:
r2_score(y_te,y_mean_pred)

0.0

In [30]:
mean_absolute_error(y_te,y_mean_pred)

11.997728016824954

Ridge Regression

In [0]:
full_information_transformer = ColumnTransformer(transformers=[
    ('units_for_sale_imputer', SimpleImputer(strategy='constant',fill_value=0),['units_for_sale']),
    ('average_rating_imputer', SimpleImputer(strategy='mean'),['average_rating']),
    ('year_encoder', OneHotEncoder(dtype=np.uint8), ['year'])
], remainder='passthrough')

In [0]:
record_store_lr_transformer = ColumnTransformer(transformers=[
    ('year_encoder', OneHotEncoder(dtype=np.uint8,handle_unknown='ignore'), ['year'])
], remainder='passthrough')

In [0]:
record_store_lr_removal_columns = [
    'market_price','units_for_sale','have','want','average_rating','rating_count','last_sold','lowest','median',
    'highest','track_titles','country','genre','style','community_have','community_want','formats','thumb_url',
    'release_url','format_description','days_since_last_sale','title',
    ]

In [0]:
from cuml import LinearRegression
from sklearn.model_selection import KFold

In [0]:
record_store_lr_pipe = Pipeline([
    ('running_time_imputer',RunningTimeImputer('running_time','number_of_tracks')),
    ('leave_one_out_encoding', LeaveOneOutEncoder(cols=['artist','label','format_text','master_id','format_name'])),
    ('record_store_column_remover', ColumnRemover(record_store_lr_removal_columns)),
    ('preprocessing',record_store_lr_transformer),
    ('scaler', StandardScaler()),
    ('lr', GridSearchCV(
        LinearRegression(),
        param_grid={
            'normalize':[False,True]
        },
        cv=KFold(n_splits=5,shuffle=True),
        scoring='neg_mean_absolute_error',
        verbose=5,
        n_jobs=-1
    ))
])

In [36]:
record_store_lr_pipe.fit(X_tr,np.log(y_tr).values)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:   23.1s remaining:   15.4s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   25.0s finished
/usr/local/lib/python3.6/site-packages/cuml/utils/input_utils.py:188: UserWarning: Expected column ('F') major order, but got the opposite. Converting data, this will result in additional memory utilization.
  warnings.warn("Expected " + order_to_str(order) + " major order, "


Pipeline(memory=None,
         steps=[('running_time_imputer',
                 RunningTimeImputer(number_of_tracks='number_of_tracks',
                                    running_time='running_time')),
                ('leave_one_out_encoding',
                 LeaveOneOutEncoder(cols=['artist', 'label', 'format_text',
                                          'master_id', 'format_name'],
                                    drop_invariant=False,
                                    handle_missing='value',
                                    handle_unknown='value', random_state=None,
                                    return_df=True, sigm...
                              estimator=LinearRegression(algorithm='eig', fit_intercept=True, normalize=False, handle=<cuml.common.handle.Handle object at 0x7f9da8a4a3d0>, verbose=False, output_type='input'),
                              iid='deprecated', n_jobs=-1,
                              param_grid={'normalize': [False, True]},
           

In [37]:
record_store_lr_pipe['lr'].best_score_

-0.5719374545847429

In [38]:
lr_pred = record_store_lr_pipe.predict(X_te)

/usr/local/lib/python3.6/site-packages/cuml/utils/input_utils.py:188: UserWarning: Expected column ('F') major order, but got the opposite. Converting data, this will result in additional memory utilization.
  warnings.warn("Expected " + order_to_str(order) + " major order, "


In [39]:
r2_score(y_te,np.exp(lr_pred))

-5.609538516542583e+30

In [40]:
mean_absolute_error(y_te,np.exp(lr_pred))

181975316554850.12

In [0]:
from cuml import Ridge

In [0]:
record_store_ridge_pipe = Pipeline([
    ('running_time_imputer',RunningTimeImputer('running_time','number_of_tracks')),
    ('leave_one_out_encoding', LeaveOneOutEncoder(cols=['artist','label','format_text','format_name','master_id'])),
    ('record_store_column_remover', ColumnRemover(record_store_lr_removal_columns)),
    ('preprocessing',record_store_lr_transformer),
    ('scaler', StandardScaler()),
    ('ridge', GridSearchCV(
        Ridge(normalize=False,solver='eig'),
        param_grid={
            'alpha': np.linspace(200,300,10),
        },
        cv=KFold(n_splits=5,shuffle=True),
        scoring='neg_mean_absolute_error',
        verbose=5,
        n_jobs=-1
    ))
])

In [43]:
record_store_ridge_pipe.fit(X_tr,np.log(y_tr).values)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   10.0s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   31.9s finished
/usr/local/lib/python3.6/site-packages/cuml/utils/input_utils.py:188: UserWarning: Expected column ('F') major order, but got the opposite. Converting data, this will result in additional memory utilization.
  warnings.warn("Expected " + order_to_str(order) + " major order, "


Pipeline(memory=None,
         steps=[('running_time_imputer',
                 RunningTimeImputer(number_of_tracks='number_of_tracks',
                                    running_time='running_time')),
                ('leave_one_out_encoding',
                 LeaveOneOutEncoder(cols=['artist', 'label', 'format_text',
                                          'format_name', 'master_id'],
                                    drop_invariant=False,
                                    handle_missing='value',
                                    handle_unknown='value', random_state=None,
                                    return_df=True, sigm...
                              estimator=Ridge(alpha=1.0, solver='eig', fit_intercept=True, normalize=False, handle=<cuml.common.handle.Handle object at 0x7f9dabc16cf0>, output_type='input'),
                              iid='deprecated', n_jobs=-1,
                              param_grid={'alpha': array([200.        , 211.11111111, 222.22222222, 

In [44]:
record_store_ridge_pipe['ridge'].best_params_

{'alpha': 266.66666666666663}

In [45]:
ridge_pred = record_store_ridge_pipe.predict(X_te)

/usr/local/lib/python3.6/site-packages/cuml/utils/input_utils.py:188: UserWarning: Expected column ('F') major order, but got the opposite. Converting data, this will result in additional memory utilization.
  warnings.warn("Expected " + order_to_str(order) + " major order, "


In [46]:
r2_score(y_te,np.exp(ridge_pred))

0.16087739710076998

In [47]:
mean_absolute_error(y_te,np.exp(ridge_pred))

8.655771833756281

## Random Forest

In [0]:
from sklearn.ensemble import RandomForestRegressor

In [0]:
random_forest_param_grid = {
    'n_trees': np.linspace(50,350,50),
    'max_depth': np.linspace(10,30,5)
}

In [0]:
record_store_random_forest_removal_columns = [
    'market_price','units_for_sale','have','want','average_rating','rating_count','last_sold','lowest','median',
    'highest','track_titles','country','genre','style','community_have','community_want','formats','thumb_url',
    'release_url','format_description','days_since_last_sale','title'
    ]
record_store_random_forest_removal_columns.extend(
    column_store._indicator
)

In [0]:
full_information_transformer = ColumnTransformer(transformers=[
    ('units_for_sale_imputer', SimpleImputer(strategy='constant',fill_value=0),['units_for_sale']),
    ('average_rating_imputer', SimpleImputer(strategy='mean'),['average_rating']),
    ('year_encoder', OneHotEncoder(dtype=np.uint8), ['year'])
], remainder='passthrough')

In [0]:
class IndicatorReducer(BaseEstimator,TransformerMixin):
    def __init__(self, indicators, algorithm,components,reduced_column_prefix= 'indicator_reduced'):
        self.indicators = indicators
        self.algorithm = algorithm
        self.components = components
        self.reduced_column_prefix = reduced_column_prefix
  
    def fit(self,X,y=None):
        return self

    def transform(self,X,y=None):
        X = X.copy()

        algorithm = self.algorithm(n_components=self.components)

        reduced_indicators = pd.DataFrame(
            algorithm.fit_transform(X.loc[:,self.indicators].values),
            index=X.index,
            columns=['_'.join([self.reduced_column_prefix,str(i)]) for i in range(self.components)]
            )

        return pd.concat([X, reduced_indicators],axis=1)

In [0]:
from cuml import RandomForestRegressor

In [71]:
random_forest_pipe = Pipeline([
    ('running_time_imputer',RunningTimeImputer('running_time','number_of_tracks')),
    ('leave_one_out_encoding', LeaveOneOutEncoder(cols=['artist','label','format_text','format_name','master_id'])),
    ('reduce_indicators', IndicatorReducer(indicators=column_store._indicator,algorithm=UMAP,components=40)),
    ('record_store_column_remover', ColumnRemover(record_store_random_forest_removal_columns)),
    ('cast_to_32', FunctionTransformer(func=lambda x: x.values.astype(np.float32))),
    ('random_forest', GridSearchCV(
        RandomForestRegressor(seed=0,accuracy_metric='mean_ae',n_bins=20,split_criterion=3,max_depth=15,n_estimators=100),
        cv=KFold(n_splits=5,shuffle=True),
        param_grid={
            'n_estimators': np.linspace(50,250,5).astype(int)
        },
        verbose=50,
        scoring='neg_mean_absolute_error',
        n_jobs=-1,
    ))
])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Setting the random seed does not fully guarantee the exact same results at this time.
  


In [0]:
df_sample = pd.concat([X_tr,y_tr],axis=1).sample(frac=0.1)
X_tr_sample = df_sample.drop('market_value',axis=1)
y_tr_sample = df_sample['market_value']

In [73]:
random_forest_pipe.fit(X_tr_sample,np.log(y_tr_sample).values.astype(np.float32))

/usr/local/lib/python3.6/site-packages/cuml/utils/input_utils.py:188: UserWarning: Expected row ('C') major order, but got the opposite. Converting data, this will result in additional memory utilization.
  warnings.warn("Expected " + order_to_str(order) + " major order, "


Fitting 5 folds for each of 3 candidates, totalling 15 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   11.1s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   38.4s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:   38.8s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   39.2s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   42.8s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done   9 out of  15 | elapsed:  1.4min remaining:   57.7s
[Parallel(n_jobs=-1)]: Done  10 out of  15 | elapsed:  2.2min remaining:  1.1min
[Parallel(n_jobs=-1)]: Done  11 out of  15 | elapsed:  2.9min remaining:  1.0min


/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


[Parallel(n_jobs=-1)]: Done  12 out of  15 | elapsed:  2.9min remaining:   43.3s
[Parallel(n_jobs=-1)]: Done  13 out of  15 | elapsed:  2.9min remaining:   26.7s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  3.5min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  3.5min finished


/usr/local/lib/python3.6/site-packages/cuml/utils/input_utils.py:188: UserWarning: Expected column ('F') major order, but got the opposite. Converting data, this will result in additional memory utilization.
  warnings.warn("Expected " + order_to_str(order) + " major order, "


Pipeline(memory=None,
         steps=[('running_time_imputer',
                 RunningTimeImputer(number_of_tracks='number_of_tracks',
                                    running_time='running_time')),
                ('leave_one_out_encoding',
                 LeaveOneOutEncoder(cols=['artist', 'label', 'format_text',
                                          'format_name', 'master_id'],
                                    drop_invariant=False,
                                    handle_missing='value',
                                    handle_unknown='value', random_state=None,
                                    return_df=True, sigm...
                              estimator=RandomForestRegressor(n_estimators=100, max_depth=16, handle=<cuml.common.handle.Handle object at 0x7f9dabdba690>, max_features='auto', n_bins=20, n_streams=8, split_algo=1, split_criterion=3, bootstrap=True, bootstrap_features=False, verbose=False, min_rows_per_node=2, rows_sample=1.0, max_leaves=-1, accurac

In [74]:
random_forest_pipe['random_forest'].best_score_

-0.07984863966703415

In [75]:
random_forest_pipe['random_forest'].best_params_

{'max_depth': 15}

In [68]:
random_forest_pred = random_forest_pipe.predict(X_te)

/usr/local/lib/python3.6/site-packages/cuml/utils/input_utils.py:188: UserWarning: Expected row ('C') major order, but got the opposite. Converting data, this will result in additional memory utilization.
  warnings.warn("Expected " + order_to_str(order) + " major order, "


In [69]:
r2_score(y_te,np.exp(random_forest_pred))

-0.09741826558182365

In [70]:
mean_absolute_error(y_te,np.exp(random_forest_pred))

10.691691918642647

## Catboost

In [77]:
!pip install catboost
from catboost import CatBoostRegressor

     |████████████████████████████████| 64.4 MB 34 kB/s 
     |████████████████████████████████| 7.1 MB 44.0 MB/s 
     |████████████████████████████████| 12.4 MB 35.1 MB/s 
     |████████████████████████████████| 88 kB 6.7 MB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11430 sha256=2a216586505da240a8254c84d69e8472f067661d8172f18cdb6a92558353456c
  Stored in directory: /root/.cache/pip/wheels/ac/cb/8a/b27bf6323e2f4c462dcbf77d70b7c5e7868a7fbe12871770cf
Successfully built retrying


In [0]:
record_store_catboost_removal_columns = [
    'market_price','units_for_sale','have','want','average_rating','rating_count','last_sold','lowest','median',
    'highest','track_titles','genre','style','community_have','community_want','formats','thumb_url',
    'release_url','format_description','days_since_last_sale','title'
    ]
record_store_catboost_removal_columns.extend(
    column_store._indicator
)

In [0]:
catboost_param_grid = {
        'depth': np.linspace(1,16,5).astype(int),
        'learning_rate' : np.logspace(-4,-1,5),
        'l2_leaf_reg': np.linspace(1,50,5).astype(int),
        'iterations': np.linspace(50,250,3),
        'n_estimators': np.linspace(20,200,5).astype(int)
}

In [0]:
catboost_pipe = Pipeline([
    ('running_time_imputer',RunningTimeImputer('running_time','number_of_tracks')),
    ('reduce_indicators', IndicatorReducer(indicators=column_store._indicator-column_store._geography,algorithm=UMAP,components=40)),
    ('record_store_column_remover', ColumnRemover(record_store_catboost_removal_columns)),   
    ('grid_search', GridSearchCV(
        CatBoostRegressor(
            random_seed=0,
            cat_features=['year','format_text','master_id','format_name','artist','label','country'],
            one_hot_max_size=25,
            task_type='GPU',
            devices='0:1',
            boosting_type='Ordered',
            verbose=True            
        ),
        param_grid={'depth': np.linspace(1,16,5).astype(int)},
        n_jobs=1,
        scoring='neg_mean_absolute_error',
        verbose=50)
    )
])

In [0]:
catboost_pipe.fit(X_tr_sample,np.log(y_tr_sample))

/usr/local/lib/python3.6/site-packages/cuml/utils/input_utils.py:188: UserWarning: Expected row ('C') major order, but got the opposite. Converting data, this will result in additional memory utilization.
  warnings.warn("Expected " + order_to_str(order) + " major order, "


Streaming output truncated to the last 5000 lines.
21:	learn: 0.7317833	total: 10.6s	remaining: 7m 53s
22:	learn: 0.7301975	total: 11.1s	remaining: 7m 52s
23:	learn: 0.7293301	total: 11.2s	remaining: 7m 34s
24:	learn: 0.7273702	total: 11.7s	remaining: 7m 35s
25:	learn: 0.7267111	total: 12.2s	remaining: 7m 36s
26:	learn: 0.7246944	total: 12.7s	remaining: 7m 37s
27:	learn: 0.7231434	total: 13.2s	remaining: 7m 38s
28:	learn: 0.7214213	total: 13.7s	remaining: 7m 38s
29:	learn: 0.7206272	total: 14.2s	remaining: 7m 38s
30:	learn: 0.7197782	total: 14.7s	remaining: 7m 38s
31:	learn: 0.7186023	total: 15.2s	remaining: 7m 39s
32:	learn: 0.7174642	total: 15.7s	remaining: 7m 39s
33:	learn: 0.7161330	total: 16.2s	remaining: 7m 39s
34:	learn: 0.7154650	total: 16.7s	remaining: 7m 40s
35:	learn: 0.7144192	total: 17.2s	remaining: 7m 39s
36:	learn: 0.7138209	total: 17.7s	remaining: 7m 39s
37:	learn: 0.7134580	total: 17.8s	remaining: 7m 29s
38:	learn: 0.7126828	total: 18.3s	remaining: 7m 30s
39:	learn: 0.

In [1]:
catboost_pipe.best_score_

NameError: ignored

In [0]:
catboost_pipe.best_params_

In [0]:
catboost_pred = catboost_pipe.predict(X_te)

In [0]:
r2_score(y_te,np.exp(catboost_pred))

In [0]:
mean_absolute_error(y_te,np.exp(catboost_pred))